In [18]:
from tqdm import tqdm
from models import CompletionNetwork, ContextDiscriminator, GlobalDiscriminator_P
from datasets2 import ImageDataset
from losses import completion_network_loss, completion_network_loss_P
from noise import AddGaussianNoise
from torch.utils.data import DataLoader
from torch.optim import Adadelta, Adam
from torch.nn import BCELoss, DataParallel
from torchvision.utils import save_image
import torchvision.utils as vutils
from PIL import Image
import torchvision.transforms as transforms
import torch
import random
import os
import argparse
import numpy as np
import json
import time

import cv2
import matplotlib.pyplot as plt
import glob

from datasets.detectron_pro import mask_img

ImportError: /opt/conda/lib/python3.6/site-packages/detectron2/_C.cpython-36m-x86_64-linux-gnu.so: undefined symbol: _ZN3c104impl25tls_local_tensor_type_setEv

In [12]:
steps=1000

snaperiod=50



In [14]:
data_dir="/root/notebooks/final/gandata/gandata1"
result_dir="/root/notebooks/final/gandata/result"


gpu = torch.device('cuda:0')

In [10]:
# dataset

trnsfm = transforms.Compose([
    transforms.ToTensor(),
    
])

trnsfm2 = transforms.Compose([
    transforms.ToTensor(),
    AddGaussianNoise(0., 0.001)
])
print('loading dataset... (it may take a few minutes)')
train_dset = ImageDataset(data_dir, trnsfm,trnsfm2, load2meme = False)
test_dset = ImageDataset(data_dir, trnsfm,trnsfm2, load2meme = False)
train_loader = DataLoader(train_dset, batch_size=16, shuffle=True)

loading dataset... (it may take a few minutes)


In [11]:
# Create model G

model_G = CompletionNetwork()
model_G = DataParallel(model_G)
opt_G = Adam(model_G.parameters())
model_G = model_G.to(gpu)

# Create model D
model_D = GlobalDiscriminator_P((3,256,256))
model_D = DataParallel(model_D)
opt_D = Adam(model_D.parameters())
model_D = model_D.to(gpu)

bceloss = BCELoss()

In [13]:
def completion_loss(background, person, output, lefttop, height = 64 , width = 32, devices = 'cuda:0'):
    """
    * background:
        - shape: batchsize * 3 * 256 * 256
    * output:
        - shape: batchsize * 3 * 256 * 256
    """
    output_crop = torch.zeros([output.shape[0],3,128,64]).to(devices)
    person_crop = torch.zeros([output.shape[0],3,128,64]).to(devices)
    
    #mask = torch.zeros(size = output.shape).to(devices)
    batchsize = output.shape[0]
    for i in range(batchsize):
        left , top = lefttop[i][0],lefttop[i][1]
        output_crop[i,:,:,:] = output[i,:,top : top + height , left : left + width]
        person_crop[i,:,:,:] = person[i,:,top : top + height , left : left + width]
        #mask[ i ,:, top : top + height , left : left + width]  = 1

    return 0.7*mse_loss(output_crop, person_crop) + 0.3*mse_loss(output, background)

In [ ]:
def train_G(n,steps):
    cnt_bdivs = 0
    pbar = tqdm(total=steps)
    while pbar.n < steps:
        for street_img, mask_with_poeple, left_top in train_loader:
            street_img = street_img.to(gpu)
            mask_with_poeple = mask_with_poeple.to(gpu)

            input = torch.cat((street_img, mask_with_poeple), dim=1)
            output = model_G(input)
            #loss = completion_network_loss(x, output, mask)
            #loss = torch.nn.functional.mse_loss(output, street_img)
            
            loss = completion_loss(street_img, mask_with_poeple, output, left_top, height = 128 , width = 64)

            # backward
            loss.backward()
            cnt_bdivs += 1

            # optimize
            opt_cn.step()
            # clear grads
            opt_cn.zero_grad()
            # update progbar
            pbar.set_description('%d | phase 1 | train loss: %.5f' % (n,loss.cpu()))
            pbar.update()
            if pbar.n % snaperiod == 0:
                with torch.no_grad():
                    x1, x2 = sample_random_batch(test_dset, batch_size=3)
                    x1 = x1.to(gpu)
                    x2 = x2.to(gpu)

                    input = torch.cat((x1, x2), dim=1)
                    output = model_G(input)
                    completed = poisson_blend(x1, output, x2)
                    imgs = torch.cat((x1.cpu(), x2.cpu(), output.cpu(),completed.cpu()), dim=2)
                    imgpath = os.path.join(result_dir, 'phase_1', '%d_step%d.png' % (n , pbar.n))
                    model_G_path = os.path.join(result_dir, 'phase_1', '%d_model_cn_step%d' % (n , pbar.n))
                    save_image(imgs, imgpath, nrow=len(x1))
                    torch.save(model_G.state_dict(), model_G_path)
            # terminate
            if pbar.n >= steps:
                break
                
    pbar.close()
